### Load imports and run the no LID case:

In [13]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking model BEFORE adding GSI
%reset
import yaml
from swmm_objects import *
from swmm_read_cso_time_series import *
from swmm_run_cso import *
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
client = MongoClient(mongoServer)
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidZeroFile = "PWD_wingo_Wakefield_Anna_list_zero.yaml"
################################################

db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collection = 'y16m12d19_AEM_Wakefield_and_Anna_50_250'

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
starting_perm = [0,0,0,0,0,0,0]
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection,starting_perm)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
2.0 min, 56.31 sec
volume = 1948.901
Stored record 58595386c10157068d752099 for zero LID usage:  peak = 1948.45, volume = 1948.901


### Loop ALL subcatchment LID numbers over a range (using np.random):

In [19]:
# Run or continue a run:
from swmm_run_cso import *
from copy import deepcopy
import numpy as np
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collection = 'y16m12d19_AEM_Wakefield_and_Anna_50_250'
thisRunCollection = db[collection]
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
numSubcatch = 7
numLID = 2
numVariables = numSubcatch*numLID
permList = []
maxLidNumber = 30
numberOfSimulations = 100
for i in range(0,numberOfSimulations):#number of simulations
    randomPerm=np.random.randint(maxLidNumber, size=numVariables) 
    randomPerm = randomPerm.tolist()
    permList.append(randomPerm)
#print permList
runNum = 0
for perm in permList:
    runNum += 1
    print "Run Number %s" % runNum    
    print perm
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    totalAreaTreated = 0
    for i in range(0,numVariables): 
        lidNumber = perm[i] # random number of LIDs for this variable
        subArea = newRunList[i]['SubcatArea'] #area of subcatchment
        newRunList[i]['Number'] = perm[i]  # random number of lids for this variable
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection,perm)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    print volumeReduction 

    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

Run Number 1
[6, 17, 24, 28, 19, 24, 25, 25, 26, 22, 4, 27, 7, 3]
2.0 min, 54.66 sec
volume = 1906.289
42.612
Run Number 2
[29, 5, 12, 26, 5, 25, 23, 26, 3, 15, 27, 12, 2, 5]
2.0 min, 59.79 sec
volume = 1912.873
36.028
Run Number 3
[3, 10, 18, 4, 22, 24, 4, 3, 11, 2, 24, 17, 7, 10]
2.0 min, 52.56 sec
volume = 1922.778
26.123
Run Number 4
[27, 5, 17, 28, 11, 2, 23, 20, 21, 21, 28, 0, 19, 18]
2.0 min, 52.40 sec
volume = 1910.718
38.183
Run Number 5
[28, 6, 23, 0, 16, 29, 8, 29, 26, 17, 7, 16, 18, 25]
2.0 min, 54.08 sec
volume = 1909.989
38.912
Run Number 6
[17, 28, 4, 0, 4, 11, 18, 8, 29, 5, 6, 20, 28, 20]
2.0 min, 53.15 sec
volume = 1918.243
30.658
Run Number 7
[6, 13, 20, 25, 25, 10, 5, 20, 18, 22, 3, 17, 23, 10]
2.0 min, 52.74 sec
volume = 1914.179
34.722
Run Number 8
[25, 9, 28, 28, 6, 13, 28, 19, 7, 25, 10, 18, 27, 21]
2.0 min, 52.14 sec
volume = 1905.98
42.921
Run Number 9
[4, 23, 19, 1, 6, 12, 16, 7, 13, 5, 7, 15, 5, 20]
2.0 min, 53.99 sec
volume = 1923.846
25.055
Run Number 10
[1

### To display all of the runs from the database:

In [17]:
print thisRunCollection

Collection(Database(MongoClient('aws-2.greenphilly.net', 27017), u'jcohen4'), u'y16m12d20_AEM_Wingo_Zero_LID_Check')


In [4]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

{u'outflow_series': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

### To Reconstitute the SWMM input file for any of the runs above

In [4]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
#print swmmInputFileStr
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()